In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import pandas as pd


In [3]:
csv_list = ["mcq_sing_cpa.csv","mcq_sing_lbk.csv","mcq_sing_nje.csv","mcq_sing_pae.csv",
           "mcq_sing_pfe.csv","mcq_sing_ungee.csv"]
csv_list_mul = ["mcq_mult_cpa.csv","mcq_mult_nje.csv","mcq_mult_pae.csv","mcq_mult_ungee.csv"]

### base model

In [3]:
model_name = "../model/Qwen2.5-3B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Warn(72319:139897580648256:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(72319:139897580648256:libvgpu.c:902)]: Initialized
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards:   0%|                     | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards: 100%|█████████████| 2/2 [00:03<00:00,  1.66s/it]
[4pdvGPU Warn(72404:140421367076672:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72404:140421367076672:hook.c:475)]: remap handles for device 1


In [4]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content[0] ==  df["output"][i]:
            acc += 1    
        answer.append(content[0])
    print(acc)
persentage = acc/total_len
print(persentage)

134
340
583
638
766
991
0.6128633271490415


In [5]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

True

### cot_model

In [6]:
model_name = "../model/law_3b_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21152137216 total=25217466368 limit=25769803776 usage=4003595264
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21745631232 total=25217466368 limit=25769803776 usage=3410101248
Loading checkpoint shards:   0%|                     | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21152137216 total=25217466368 limit=25769803776 usage=4003595264
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21745631232 total=25217466368 limit=25769803776 usage=3410101248
Loading checkpoint shards: 100%|█████████████| 2/2 [00:04<00:00,  2.42s/it]
[4pdvGPU Warn(72500:140335937480512:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72500:140335937480512:hook.c:475)]: remap handles for device 1


In [7]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content[0] ==  df["output"][i]:
            acc += 1    
        answer.append(content[0])
    print(acc)
persentage = acc/total_len
print(persentage)

134
336
576
631
762
983
0.6079158936301794


In [8]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

True

### full_model

In [9]:
model_name = "../model/law_3b_full_data_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=17442275328 total=25217466368 limit=25769803776 usage=7713457152
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=19254214656 total=25217466368 limit=25769803776 usage=5901517824
Loading checkpoint shards:   0%|                                        | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=17442275328 total=25217466368 limit=25769803776 usage=7713457152
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=19254214656 total=25217466368 limit=25769803776 usage=5901517824
Loading checkpoint shards: 100%|████████████████████████████████| 2/2 [00:03<00:00,  1.75s/it]
[4pdvGPU Warn(72631:139770765186880:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72631:139770765186880:hook.c:475)]: remap handles for device 1


In [9]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content[0] ==  df["output"][i]:
            acc += 1    
        answer.append(content[0])
    print(acc)
persentage = acc/total_len
print(persentage)

145
363
607
661
799
1031
0.6376004947433519


In [18]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    print(len(df))


197
275
537
118
170
320


In [13]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

True

### 7b-lora

In [33]:
model_name = "../model/law_7b_full_data_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(93520:139710384940864:memory.c:566)]: orig free=18071420928 total=25217466368 limit=25769803776 usage=7084311552
[4pdvGPU Msg(93520:139710384940864:memory.c:566)]: orig free=16223830016 total=25217466368 limit=25769803776 usage=8931902464
Loading checkpoint shards:   0%|                                        | 0/4 [00:00<?, ?it/s][4pdvGPU Msg(93520:139710384940864:memory.c:566)]: orig free=18071420928 total=25217466368 limit=25769803776 usage=7084311552
[4pdvGPU Msg(93520:139710384940864:memory.c:566)]: orig free=16223830016 total=25217466368 limit=25769803776 usage=8931902464
Loading checkpoint shards: 100%|████████████████████████████████| 4/4 [00:06<00:00,  1.73s/it]
[4pdvGPU Warn(94274:139930420418368:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(94274:139930420418368:hook.c:475)]: remap handles for device 1


In [34]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项\(只需要回答正确选项即可，不需要进行分析,示例回答：B\)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content ==  df["output"][i]:
            acc += 1    
        answer.append(content)
    print(acc)
persentage = acc/total_len
print(persentage)

166
400
727
807
963
1232
0.7619047619047619


In [35]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

True

### base_model 7b

In [6]:
model_name = "../model/Qwen2.5-7B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(93133:139883438561088:memory.c:566)]: orig free=18128044032 total=25217466368 limit=25769803776 usage=7027688448
[4pdvGPU Msg(93133:139883438561088:memory.c:566)]: orig free=15261237248 total=25217466368 limit=25769803776 usage=9894495232
Loading checkpoint shards:   0%|                                        | 0/4 [00:00<?, ?it/s][4pdvGPU Msg(93133:139883438561088:memory.c:566)]: orig free=18128044032 total=25217466368 limit=25769803776 usage=7027688448
[4pdvGPU Msg(93133:139883438561088:memory.c:566)]: orig free=15261237248 total=25217466368 limit=25769803776 usage=9894495232
Loading checkpoint shards: 100%|████████████████████████████████| 4/4 [00:07<00:00,  1.91s/it]
[4pdvGPU Warn(93288:140320905901888:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(93288:140320905901888:hook.c:475)]: remap handles for device 1


In [30]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项\(只需要回答正确选项即可，不需要进行分析,示例回答:A\)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content ==  df["output"][i]:
            acc += 1    
        answer.append(content)
    print(acc)
persentage = acc/total_len
print(persentage)

154
367
657
713
860
1107
0.6846011131725418


In [32]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

False

### mult_7b_base

In [4]:
model_name = "../model/Qwen2.5-7B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Warn(94946:139773074536256:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(94946:139773074536256:libvgpu.c:902)]: Initialized
[4pdvGPU Msg(94946:139773074536256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(94946:139773074536256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards:   0%|                                        | 0/4 [00:00<?, ?it/s][4pdvGPU Msg(94946:139773074536256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(94946:139773074536256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards: 100%|████████████████████████████████| 4/4 [00:07<00:00,  1.83s/it]
[4pdvGPU Warn(95034:140643644811072:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(95034:140643644811072:hook.c:475)]: remap handles for device 1


In [10]:
csv_list_mul = ["mcq_mult_cpa.csv","mcq_mult_nje.csv","mcq_mult_pae.csv","mcq_mult_ungee.csv"]
path = "../data/val_csv_mul/"
total_len=0
answer = []
acc = 0
for name in csv_list_mul:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的多项选择题,请根据题目选择正确的选项\(只需要回答正确选项即可，不需要进行分析,回答格式：ABC\)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if str(content) ==  str(df["output"][i]):
            acc += 1    
        answer.append(content)
    print(acc)
persentage = acc/total_len
print(persentage)

76
205
302
347
0.3668076109936575


In [12]:
def validate_list_elements(input_list):
    """
    判断列表中的元素是否只包含由A、B、C、D组成的非空字符串
    
    参数:
        input_list: 要检查的列表
        
    返回:
        bool: 如果所有元素都是非空字符串且只包含A、B、C、D字符则返回True，否则返回False
    """
    # 首先检查是否为空列表
    if not input_list:
        return False
    
    allowed_chars = {'A', 'B', 'C', 'D'}
    
    for item in input_list:
        # 检查是否为字符串类型
        if not isinstance(item, str):
            return False
        
        # 检查字符串是否为空
        if not item:
            return False
        
        # 检查字符串中的每个字符是否都在允许的字符集中
        for char in item:
            if char not in allowed_chars:
                return False
                
    return True




validate_list_elements(answer)

False

### mult_full_7b

In [4]:
model_name = "../model/law_7b_full_data_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Warn(95540:140625019139904:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(95540:140625019139904:libvgpu.c:902)]: Initialized
[4pdvGPU Msg(95540:140625019139904:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(95540:140625019139904:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards:   0%|                                        | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(95540:140625019139904:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(95540:140625019139904:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards: 100%|████████████████████████████████| 2/2 [00:02<00:00,  1.47s/it]
[4pdvGPU Warn(95625:140206313310016:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(95625:140206313310016:hook.c:475)]: remap handles for device 1


In [5]:
csv_list_mul = ["mcq_mult_cpa.csv","mcq_mult_nje.csv","mcq_mult_pae.csv","mcq_mult_ungee.csv"]
path = "../data/val_csv_mul/"
total_len=0
answer = []
acc = 0
for name in csv_list_mul:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的**多项选择题**,请根据题目选择正确的选项\(只需要回答正确选项即可，不需要进行分析,**回答格式：ABC**\)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if str(content) ==  str(df["output"][i]):
            acc += 1    
        answer.append(content)
    print(acc)
persentage = acc/total_len
print(persentage)

13
17
24
28
0.02959830866807611


In [4]:
answer

['ABD',
 'ABCD',
 'AC',
 'A',
 'AB',
 'AC',
 'ACD',
 'BC',
 'ABD',
 'ACD',
 'ABCD',
 'A',
 'ABC',
 'A,C',
 'ACD',
 'AD',
 'AD',
 'ABC',
 'ABD',
 'BC',
 'BCD',
 'AD',
 'ABCD',
 'A,B,C,D',
 'A,B,C',
 'ABCD',
 'ABC',
 'A',
 'ABCD',
 'A',
 'AD',
 'B',
 'A',
 'BD',
 'AD',
 'ACD',
 'ACD',
 'ABD',
 'ABCD',
 'AB',
 'A',
 'A,B,D',
 'ACD',
 'ABC',
 'BCD',
 'ABCD',
 'BCD',
 'ABC',
 'BD',
 'ABC',
 'ACD',
 'ABD',
 'ABCD',
 'CD',
 'AB',
 'BCD',
 'A',
 'ABD',
 'A,B,D',
 'BCD',
 'CD',
 'ACD',
 'AD',
 'D',
 'ABCD',
 'BC',
 'BD',
 'ABD',
 'C',
 'BCD',
 'ABCD',
 'ABCD',
 'ABCD',
 'A',
 'ABCD',
 'A',
 'AB',
 'A/B/C',
 'A',
 'BD',
 'A',
 'ACD',
 'AB',
 'BD',
 'B',
 'AB',
 'ABC',
 'ABC',
 'ABD',
 'BCD',
 'BD',
 'ABC',
 'ACD',
 'ACD',
 'ABC',
 'A',
 'BCD',
 'C',
 'A,D',
 'ACD',
 'A,C',
 'B',
 'ABC',
 'ACD',
 'CD',
 'ABD',
 'BCD',
 'A',
 'ABCD',
 'ACD',
 'ACD',
 'C',
 'AC',
 'A',
 'AD',
 'A',
 'ABD',
 'AD',
 'ABD',
 'ABCD',
 'ABCD',
 'ACD',
 'C',
 'A',
 'ABD',
 'ACD',
 'A',
 'AB',
 'C',
 'B',
 'ABCD',
 'A',
 